In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import sys
sys.path.append("..")
import data_loader as dl

In [2]:
dl.clean_msgfplus("2ng")

,#SpecFile,SpecID,scan,FragMethod,Precursor,IsotopeError,PrecursorError(ppm),Charge,Peptide,Protein,DeNovoScore,MSGFScore,SpecEValue,EValue,QValue,probability,peptide,decoy
0,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=19653,19653,HCD,1095.45940,0,2.785833,2,NMGGPYGGGNYGPGGSGGSGGYGGR,"sp|P22626|ROA2_HUMAN(pre=R,post=S)",316,312,6.192867e-32,1.365737e-24,0.000000,0.000000,NMGGPYGGGNYGPGGSGGSGGYGGR,False
1,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=21514,21514,HCD,929.07600,0,1.116807,3,EGIC+57.021ALGGTSELSSEGTQHSYSEEEK,"sp|P13797|PLST_HUMAN(pre=K,post=Y)",223,223,1.003438e-30,2.214772e-23,0.000000,0.000000,EGICALGGTSELSSEGTQHSYSEEEK,False
2,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=41561,41561,HCD,1379.73780,0,2.211844,2,SNYLLNTTIAGVEEADVVLLVGTNPR,"sp|P28331|NDUS1_HUMAN(pre=R,post=F)",281,276,5.478810e-30,1.209274e-22,0.000000,0.000000,SNYLLNTTIAGVEEADVVLLVGTNPR,False
3,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=17561,17561,HCD,921.80240,0,4.767347,3,AQGPAASAEEPKPVEAPAANSDQTVTVK,"sp|P80723|BASP1_HUMAN(pre=K,post=E)",199,196,1.565620e-29,3.460831e-22,0.000000,0.000000,AQGPAASAEEPKPVEAPAANSDQTVTVK,False
4,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=28096,28096,HCD,689.08954,0,2.834364,4,LNC+57.021QVIGASVDSHFC+57.021HLAWVNTPK,"sp|Q06830|PRDX1_HUMAN(pre=K,post=K)",254,246,4.857852e-29,1.070362e-21,0.000000,0.000000,LNCQVIGASVDSHFCHLAWVNTPK,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35788,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=42854,42854,HCD,384.13535,1,-9.304376,2,DGSGGGGEM,"XXX_sp|Q5SRD1|TI23B_HUMAN(pre=K,post=-)",1,-85,2.153168e-02,4.572388e+05,0.479817,0.484493,DGSGGGGEM,True
35789,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=10967,10967,HCD,378.66696,1,12.188486,2,SPGGGPHM+15.995,"XXX_sp|Q86VX9|MON1A_HUMAN(pre=R,post=-)",52,-92,2.320015e-02,4.887714e+05,0.479855,0.484448,SPGGGPHM+15.995,True
35790,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=41999,41999,HCD,383.13147,-1,-10.662175,2,DGSGGGGEM,"XXX_sp|Q5SRD1|TI23B_HUMAN(pre=K,post=-)",0,-93,2.547064e-02,5.408852e+05,0.479893,0.484493,DGSGGGGEM,True
35791,Ex_Auto_DrM3_30umT4_2ngQC_60m_half.mzML,controllerType=0 controllerNumber=1 scan=43501,43501,HCD,386.65695,-1,-18.920235,2,APGNSPC+57.021A,"sp|Q68DC2|ANKS6_HUMAN(pre=R,post=-)",12,-93,2.688617e-02,5.664269e+05,0.479913,0.484493,APGNSPCA,False


In [3]:
def format_oxidation(row, column, to_replace):
    peptide = row[column]
#     print(to_replace)
    replace_with = "+15.995"
    if pd.isna(peptide):
        new_pep = peptide
    else:
        if to_replace in peptide:
            new_pep = peptide.replace(to_replace, replace_with)
        else:
            new_pep = peptide
    return new_pep

def format_carbamidomethyl(row, column, to_replace):
    peptide = row[column]
#     print(to_replace)
    replace_with = ""
    if pd.isna(peptide):
        new_pep = peptide
    else:
        if to_replace in peptide:
            new_pep = peptide.replace(to_replace, replace_with)
        else:
            new_pep = peptide
    return new_pep



In [4]:
def clean_MM(input_file):
    data = pd.read_csv(input_file, sep = '\t')

    #make a new col that includes modifide peptides
    data['temp_peptide'] = data.apply(lambda row: format_oxidation(row, "Full Sequence", "[Common Variable:Oxidation on M]"), axis=1)
    data["temp2"] = data.apply(lambda row: format_carbamidomethyl(row, "temp_peptide", "[Common Fixed:Carbamidomethyl on C]"), axis=1)

    data = data.replace({"Decoy": {'Y': True, 'N': False}})
    #uniform naming
    data_new = data.rename({"Decoy": "decoy", "Scan Number": "scan", "temp2": "peptide", 'PEP_QValue': 'probability'}, axis=1)
    data_new = data_new.filter((['decoy', 'scan', 'peptide','probability', 
                                 "Matched Ion Series", 'Matched Ion Intensities', 'Matched Ion Counts', 'Matched Ion Mass Diff (Ppm)']))

    return data_new



In [5]:
def filter_data(df, SC):
    df = df[df["decoy"]==False]
    df = df.sort_values("probability")
    df = df.drop_duplicates(subset=["scan"], keep="first") #keep highest scoring
    df = df.sort_values("probability")
    df = df.drop_duplicates(subset=["peptide"], keep="first")#I only want unique peptides 
    if SC==True: 
        df = df[(df["probability"] <= .01)]
    else:
        df = df[df["probability"] <= .01]
    return df



In [6]:
small = clean_MM("~/Payne/bulk_data_vs_2ng/data_from_pride/OR11_20160122_PG_HeLa_CVB3_CT_A.raw")
large = clean_MM("~/Payne/bulk_data_vs_2ng/metamorpheus_output/AllPSMs2.psmtsv")

small = filter_data(small, SC=True)
large = filter_data(large, SC=False)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa1 in position 1: invalid start byte